In [2]:
library(mitml)
library(dplyr)
library(lme4)
library(mice)
library(emmeans)
options(emmeans_msg = FALSE)
# Load the trial posteriors data
df_orig <- read.csv("files/trial_posteriors.csv")

# Display first few rows
head(df_orig)

*** This is beta software. Please report any bugs!
*** See the NEWS file for recent changes.


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'lme4' was built under R version 4.4.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 4.4.3"

Attaching package: 'mice'


The following object is masked from 'package:stats':

    filter


The following objects are masked from 'package:base':

    cbind, rbind


Warning message:
"package 'emmeans' was built under R version 4.4.3"
Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'



,participant,epoch,condition,rt_t1,correct_t1,prob_A,prob_B,prob_C,prob_D,prob_E,prob_F
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,VP19,473,short,0.6328125,1,0.034,0.464,0.208,0.158,0.060,0.007
2,VP19,1139,short,0.8925781,1,0.014,0.329,0.404,0.050,0.038,0.036
3,VP19,850,short,0.7031250,1,0.002,0.207,0.465,0.075,0.127,0.044
4,VP8,1061,short,0.5781250,1,0.005,0.381,0.189,0.199,0.087,0.032
5,VP9,1258,short,0.6542969,1,0.004,0.182,0.281,0.172,0.179,0.048
6,VP13,1210,short,0.5078125,1,0.053,0.565,0.221,0.035,0.018,0.005


In [3]:
df <- df_orig %>% filter(condition == "short")

In [4]:
# Then extract and normalize probability columns
prob_cols = grep("^prob_", names(df), value = TRUE)
prob_matrix = as.matrix(df[, prob_cols])
prob_matrix <- prob_matrix / rowSums(prob_matrix)

# Update df with normalized probabilities
df[, prob_cols] <- prob_matrix

# Remove rows where normalization introduced NAs (keep df and prob_matrix in sync)
na_rows <- apply(prob_matrix, 1, function(x) any(is.na(x)))
df <- df[!na_rows, ]
prob_matrix <- prob_matrix[!na_rows, ]

In [5]:
M <- 100
set.seed(42)

sequence_levels <- c("A", "B", "C", "D", "E", "F")

imputeds <- vector("list", M)

for (i in 1:M) {
    sampled_seq <- apply(prob_matrix, 1, function(p) {
        sample(sequence_levels, size = 1, prob = p)
    })

    df_imp <- df %>%
        mutate(sequence = sampled_seq) %>%
        select(participant, rt_t1, correct_t1, sequence)

    imputeds[[i]] <- df_imp
}

In [6]:
fits_rt <- lapply(imputeds, function(df_imp) {
    lmer(rt_t1 ~ sequence + (1 | participant), data = df_imp)
})

fits_acc <- lapply(imputeds, function(df_imp) {
    glmer(correct_t1 ~ sequence + (1 | participant), data = df_imp, family = binomial)
})

In [7]:
# Pool the results using Rubin's rules
pooled_rt <- testEstimates(fits_rt, extra.pars = TRUE)
print(pooled_rt)

pooled_acc <- testEstimates(fits_acc, extra_pars = TRUE)
print(pooled_acc)


Call:

testEstimates(model = fits_rt, extra.pars = TRUE)

Final parameter estimates and inferences obtained from 100 imputed data sets.

             Estimate Std.Error   t.value        df   P(>|t|)       RIV       FMI 
(Intercept)     0.440     0.017    26.197 2.054e+06     0.000     0.007     0.007 
sequenceB       0.098     0.004    24.417 1.658e+03     0.000     0.323     0.245 
sequenceC       0.188     0.005    34.268 5.909e+02     0.000     0.693     0.411 
sequenceD       0.210     0.007    30.268 4.938e+02     0.000     0.811     0.450 
sequenceE       0.321     0.009    37.523 2.843e+02     0.000     1.440     0.593 
sequenceF       0.378     0.014    26.277 2.032e+02     0.000     2.312     0.701 

                                 Estimate 
Intercept~~Intercept|participant    0.006 
Residual~~Residual                  0.019 
ICC|participant                     0.227 

Unadjusted hypothesis test as appropriate in larger samples.


Call:

testEstimates(model = fits_acc, extra

In [43]:
b <- coef(pooled_rt)   # or coef(pooled_rt) depending on object
V <- vcov(pooled_rt)       # or vcov(pooled_rt)

# Order must match b / V
# sequences: A (ref), B, C, D, E, F
L <- rbind(
  A = c(1, 0, 0, 0, 0, 0),
  B = c(1, 1, 0, 0, 0, 0),
  C = c(1, 0, 1, 0, 0, 0),
  D = c(1, 0, 0, 1, 0, 0),
  E = c(1, 0, 0, 0, 1, 0),
  F = c(1, 0, 0, 0, 0, 1)
)

mu <- as.vector(L %*% b)
se <- sqrt(diag(L %*% V %*% t(L)))

crit <- qnorm(0.975)  # or qt(df, ...) if you want MI df

rt_lo <- mu - crit * se
rt_hi <- mu + crit * se
rt_df <- data.frame(sequence = rownames(L), rt_mean = mu, rt_se = se, rt_lo = rt_lo, rt_hi = rt_hi)

In [48]:
b <- coef(pooled_acc)   # or coef(pooled_rt) depending on object
V <- vcov(pooled_acc)       # or vcov(pooled_rt)

# Order must match b / V
# sequences: A (ref), B, C, D, E, F
L <- rbind(
  A = c(1, 0, 0, 0, 0, 0),
  B = c(1, 1, 0, 0, 0, 0),
  C = c(1, 0, 1, 0, 0, 0),
  D = c(1, 0, 0, 1, 0, 0),
  E = c(1, 0, 0, 0, 1, 0),
  F = c(1, 0, 0, 0, 0, 1)
)

eta <- as.vector(L %*% b)
se_eta <- sqrt(diag(L %*% V %*% t(L)))

inv_logit <- function(x) plogis(x)

crit <- qnorm(0.975)  # or use qt() with pooled df if you want

lo_eta <- eta - crit * se_eta
hi_eta <- eta + crit * se_eta

p  <- inv_logit(eta)
lo <- inv_logit(lo_eta)
hi <- inv_logit(hi_eta)

acc_out <- data.frame(
  sequence = rownames(L),
  logit = eta,
  logit_se = se_eta,
  prob = p,
  prob_lo = lo,
  prob_hi = hi
)
res_df <- merge(rt_df, acc_out)

In [50]:
write.csv(res_df, "files/behavior_plot_results.csv", row.names = FALSE)

### Omnibus test: Main effect of sequence

In [51]:
# Omnibus test for RT: Test all sequence coefficients jointly
# This tests: sequenceB = sequenceC = sequenceD = sequenceE = sequenceF = 0
omnibus_rt <- testConstraints(fits_rt, constraints = c("sequenceB", "sequenceC", "sequenceD", "sequenceE", "sequenceF"), method = "D1")
print(omnibus_rt)


Call:

testConstraints(model = fits_rt, constraints = c("sequenceB", 
    "sequenceC", "sequenceD", "sequenceE", "sequenceF"), method = "D1")

Hypothesis test calculated from 100 imputed data sets. The following
constraints were specified:

                Estimate Std. Error 
   sequenceB:      0.098      0.005 
   sequenceC:      0.188      0.007 
   sequenceD:      0.210      0.008 
   sequenceE:      0.321      0.009 
   sequenceF:      0.378      0.012 

Combination method: D1 

     F.value      df1      df2    P(>F)      RIV 
     448.907        5 1415.536    0.000    1.432 

Unadjusted hypothesis test as appropriate in larger samples.



In [ ]:
# Omnibus test for Accuracy
omnibus_acc <- testConstraints(fits_acc, constraints = c("sequenceB", "sequenceC", "sequenceD", "sequenceE", "sequenceF"), method = "D1")
print(omnibus_acc)

### Pairwise contrasts

In [46]:
# Generate all pairwise contrasts for RT
seq_levels <- c("A", "B", "C", "D", "E", "F")
pairs_idx <- combn(1:6, 2)

# Function to create constraint string
make_constraint <- function(i, j, seq_levels) {
  # With treatment coding, A=intercept (position 1), B-F are positions 2-6
  if (i == 1) {
    # Comparing A vs another
    return(paste0("0 - sequence", seq_levels[j]))
  } else if (j == 1) {
    # Comparing another vs A
    return(paste0("sequence", seq_levels[i], " - 0"))
  } else {
    # Comparing two non-reference levels
    return(paste0("sequence", seq_levels[i], " - sequence", seq_levels[j]))
  }
}

# Build all constraints
constraints_rt <- apply(pairs_idx, 2, function(pair) {
  make_constraint(pair[1], pair[2], seq_levels)
})

# Label constraints
contrast_labels <- apply(pairs_idx, 2, function(pair) {
  paste0(seq_levels[pair[1]], " - ", seq_levels[pair[2]])
})
names(constraints_rt) <- contrast_labels

# Test each constraint and collect results
rt_contrasts <- lapply(seq_along(constraints_rt), function(k) {
  tc <- testConstraints(fits_rt, constraints = constraints_rt[k], method = "D1")
  result <- as.data.frame(tc$test)
  result$contrast <- names(constraints_rt)[k]
  return(result)
})

# Combine results
rt_contrasts_df <- do.call(rbind, rt_contrasts)
rownames(rt_contrasts_df) <- NULL

# Find the p-value column automatically
cat("Available columns:", paste(names(rt_contrasts_df), collapse=", "), "\n")
p_col <- grep("P\\.*|p\\.value|pvalue", names(rt_contrasts_df), value = TRUE, ignore.case = TRUE)[1]
cat("Using p-value column:", p_col, "\n")

# Apply Holm correction
rt_contrasts_df$p_adj <- p.adjust(rt_contrasts_df[[p_col]], method = "holm")

# Display results (contains F.value, df1, df2, P(>F), RIV, contrast, p_adj)
print(rt_contrasts_df)

Available columns: F.value, df1, df2, P(>F), RIV, contrast 
Using p-value column: P(>F) 
       F.value df1       df2         P(>F)       RIV contrast         p_adj
1   596.182734   1 1547.2371 1.197388e-111 0.3233167    A - B 1.556605e-110
2  1174.329404   1  557.5948 2.430400e-139 0.6929348    A - C 3.645600e-138
3   916.182061   1  467.3196 3.286405e-112 0.8108199    A - D 4.600966e-111
4  1408.005355   1  272.2949 1.313160e-109 1.4397722    A - E 1.575792e-108
5   690.489670   1  196.5952  3.042251e-66 2.3117212    A - F  3.042251e-65
6   299.880287   1  362.6205  2.175396e-49 1.0391062    B - C  1.957856e-48
7   278.535126   1  357.0821  1.233874e-46 1.0559734    B - D  8.637116e-46
8   754.264007   1  249.5404  2.186841e-77 1.6123579    B - E  2.405525e-76
9   391.561156   1  189.1113  6.007193e-48 2.4748556    B - F  4.805754e-47
10    8.240951   1  302.9444  4.384055e-03 1.2660241    C - D  4.384055e-03
11  190.069864   1  204.0418  5.430990e-31 2.1720004    C - E  3.258594e-30

### Pairwise contrasts for Accuracy

In [50]:
# Generate all pairwise contrasts for Accuracy (same constraints as RT)
# Test each constraint and collect results
acc_contrasts <- lapply(seq_along(constraints_rt), function(k) {
  tc <- testConstraints(fits_acc, constraints = constraints_rt[k], method = "D1")
  result <- as.data.frame(tc$test)
  result$contrast <- names(constraints_rt)[k]
  return(result)
})

# Combine results
acc_contrasts_df <- do.call(rbind, acc_contrasts)
rownames(acc_contrasts_df) <- NULL

# Find the p-value column automatically (use same as RT)
cat("Available columns:", paste(names(acc_contrasts_df), collapse=", "), "\n")
p_col <- grep("P\\.*|p\\.value|pvalue", names(acc_contrasts_df), value = TRUE, ignore.case = TRUE)[1]
cat("Using p-value column:", p_col, "\n")

# Apply Holm correction
acc_contrasts_df$p_adj <- p.adjust(acc_contrasts_df[[p_col]], method = "holm")

# Display results (contains F.value, df1, df2, P(>F), RIV, contrast, p_adj)
print(acc_contrasts_df)

Available columns: F.value, df1, df2, P(>F), RIV, contrast 
Using p-value column: P(>F) 
       F.value df1      df2        P(>F)       RIV contrast        p_adj
1  37.01462675   1 732.6365 1.888562e-09 0.5537300    A - B 2.832843e-08
2  24.64630680   1 384.7190 1.037016e-06 0.9779484    A - C 1.348120e-05
3  28.12796876   1 397.9412 1.888080e-07 0.9454252    A - D 2.643311e-06
4  21.20664052   1 518.2995 5.194210e-06 0.7385052    A - E 6.233053e-05
5   7.09977159   1 493.8583 7.961576e-03 0.7710260    A - F 8.757733e-02
6   0.99154651   1 388.0088 3.199853e-01 0.9695962    B - C 1.000000e+00
7   5.59196566   1 318.2492 1.864279e-02 1.1966859    B - D 1.864279e-01
8   3.26079555   1 476.3249 7.158587e-02 0.7967375    B - E 6.442728e-01
9   0.66442742   1 427.8711 4.154547e-01 0.8808775    B - F 1.000000e+00
10  2.56879718   1 352.9570 1.098851e-01 1.0689861    C - D 8.790805e-01
11  1.03446579   1 409.0170 3.097133e-01 0.9201839    C - E 1.000000e+00
12  0.13293116   1 367.1450 7.15622